In [10]:
# Imports
from qiime2 import Metadata
import pandas as pd
from qiime2 import Artifact, sdk
from qiime2.plugins.dada2.methods import denoise_pyro # The samples were obtained through pyrosequencing
import qiime2.plugins.metadata.actions as metadata_actions
from qiime2.plugins.feature_classifier.pipelines import classify_consensus_blast
import qiime2.plugins.feature_table.methods as ftm
import qiime2.plugins.feature_classifier.actions as feature_classifier_actions
from qiime2.plugins.feature_table.visualizers import tabulate_seqs
import qiime2.plugins.feature_table.actions as fta
import matplotlib.pyplot as plt
import matplotlib 
import seaborn as sns
from Bio import SeqIO
%matplotlib inline

matplotlib.use('module://ipykernel.pylab.backend_inline')
pm = sdk.PluginManager()
def see(artifact):
    from_format = artifact.format
    if issubclass(from_format, sdk.plugin_manager.SingleFileDirectoryFormatBase):
        from_format = artifact.format.file.format
    return set(pm.transformers[from_format].keys())
import os
import pandas
import qiime2
import tempfile

def v2frame(viz_fp: str) -> list:
    '''viz_fp is a path to the qiime2 visualization object'''
    viz = qiime2.Visualization.load(viz_fp)
    with tempfile.TemporaryDirectory() as tmpdir:
        viz.export_data(tmpdir)
        fp = os.path.join(tmpdir, 'quality-plot.html')
        ov = os.path.join(tmpdir, 'overview.html')
        dfs = pandas.read_html(fp, index_col=0)
        df2s = pandas.read_html(ov, index_col=0)
    return dfs + df2s

# Importing data from fastq
Qiime2 uses a compressed type of file format called an 'Artifact' for its analyses. Artifacts have different semantic types e.g. `FeatureData[Sequence]`, `Phylogeny[Unrooted]` depending on the type of data they contain. To begin the analysis, I need to import my fastq files into `FeatureData[SequencesWithQuality]` or `FeatureData[PairedEndSequencesWithQuality]`. 

Although all of these reads were prepared with Illumina devices, sequencing quality can vary between sequencing centres, meaning that each sample will likely need specific parameters for cleaning. Furthermore, two of the samples were sequenced with paired-end format. This means that I'll need to import each sample into five different artifacts, merging them together once they have been cleaned

Quality $(Q)$ is commonly measured in Phred scores, denoted as  $Q=-10 \log_{10}P$, where $P$ is the probability of an incorrect base call. Therefore, higher values for Phred indicate a lower probability of an erroneous base. Every base position is given a Phred score, and it is common to see the score decrease the longer the read 

```bash
# Relevant commands
qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path devon.tsv \
  --output-path devonFQ.qza \
  --input-format PairedEndFastqManifestPhred64V2

qiime tools import \
  --type 'SampleData[SequencesWithQuality]' \
  --input-path neem.tsv \
  --output-path neem.qza \
  --input-format SingleEndFastqManifestPhred33V2
```

# Initial exploration
To get an idea of each sample's read quality, I used external tools: `FastQC` paired with `MultiQC`. I was mainly looking for the 

```bash
for sample in {Barrow_Alaska,Devon_ice_cap,Mutzagh_Ata,Greenland_ice_sheet,Neem}
    do 
        fastqc ${sample}/*
        multiqc $sample -f
    done
```

For j

In [14]:
quality = pd.read_csv('data/fastqc_per_base_sequence_quality_plot.tsv', sep='\t')
# fig, ax = plt.subplots()
# for i in quality.columns:
#     ax.scatter(x = quality.index, y = quality[i])
# plt.show()

,Position (bp),SRR21738726,SRR21738727,SRR21738728,SRR21738729,SRR21738730,SRR21738731,SRR21738732,SRR21738733,SRR21738734,...,SRR21738774,SRR21738775,SRR21738776,SRR21738777,SRR21738778,SRR21738779,SRR21738780,SRR21738781,SRR21738782,SRR21738783
0,1,32.556464,32.513028,32.618960,31.952148,36.769341,36.181801,32.457361,32.998586,36.282621,...,32.423528,32.437997,32.109506,32.723858,32.669556,32.620296,32.309986,32.436535,32.098632,32.421804
1,2,32.340534,32.055915,32.912858,32.703985,36.574296,36.464845,32.386089,32.392373,36.549014,...,32.836605,32.677679,32.484029,32.364465,32.436932,32.252363,32.818052,32.644565,32.375321,32.704913
2,3,32.676928,33.006785,32.420093,32.085292,36.115464,36.451733,32.810785,32.784026,36.446220,...,32.417235,32.868715,32.236824,32.793034,32.669621,33.047794,32.324027,32.850439,32.131472,32.572303
3,4,32.943249,32.493961,32.458894,32.749672,37.070779,36.970764,33.026628,32.944820,37.089327,...,32.638480,32.946150,32.531195,32.990324,32.833054,33.038921,32.603378,32.734268,32.767295,32.650283
4,5,33.127655,33.139903,32.874607,32.308082,36.836939,35.804727,33.201336,33.289707,35.886218,...,32.981249,32.919145,32.864025,33.025866,33.240044,33.101581,33.014091,33.067972,32.932185,33.045927
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,451,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,34.000000,NaN,NaN,NaN,NaN
146,454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147,464,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148,474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Data cleaning
- Ordinarily, we clean fastq data by trimming the reads to a length with relatively high quality scores (e.g. the average quality at base 200+ is less than 30, so we trim to keep only the first 200 bases from the 5' end). Unfortunately, several of the samples in each site exhibit varying mean quality scores at the same base position. In addition to using a read trimmer, `dada2`, I will process the reads with a quality filter to retain high-confidence hits. An adapter filter will also be used because of the adapter content in some samples.  
- The read lengths I've trimmed to are listed in the `samplelist.tsv` file, along with the sample names

In [ ]:
# Load the artifacts


# Sequence clustering

# Taxonomic analyses
For sample taxonomic classification, I will be trying out all three of the methods available in qiime2: 

#### Database preparation
Two databases were obtained:

- All annotated 16S rRNA from the NCBI ftp [website](https://ftp-ncbi-nlm-nih-gov.ejournal.mahidol.ac.th/blast/db/v5/)
- Annotated 16S rRNA used by the tool [MicFunPred](https://github.com/microDM/MicFunPred)
- A 16S rRNA database from [EzBioCloud](https://www.ezbiocloud.net/dashboard)

The MicFunPred and NCBI databases were obtained in the standard BLAST format, and need to be converted into compatible data types for import into the qiime2 workflow. Specficially, I needed to convert them into FASTA format with an associated taxonomy mapping file (in HeaderlessTSVTaxonomyFormat)
- Steps
    - 1 Extract all entries from the database in FASTA format
    - 2 Extract the header and convert it into the HeaderLessTSVTaxonomyFormat, which is a tab-delimited file of FASTA identifiers followed by their taxonomic assignments
    - 3 Remove the taxonomic assignments from the original FASTA file
    - 4 Concatenate the respective files types together, then import them files as qiime2 Artifacts
    - 5 Repeat for the other database (if both databases used the identifier conventions I could have combined them and processed them together but unfortunately this was not the case )

```bash
# 1
blastdbcmd -db NCBI_16S/16S_ribosomal_RNA -entry all > NCBI_16S.fasta
blastdbcmd -db micfun/micfun16S -entry all > micfun.fasta

# 2 
grep '>' NCBI_16S.fasta | tr -d '>' | sed 's/ /\t/' | sed 's/ /_/g' > NCBI_16SID.txt
grep '>' micfun.fasta | tr -d '>' | sed 's/_/\t/' | sort | uniq > micfunID.txt # Unfortunately several of the headers repeat 

# 3 
cat micfun.fasta | sed 's/_.*//' > micfunID.fasta
cat NCBI_16S.fasta | sed 's/ .*//' > ncbi16sID.fasta

# 4 
cat micfunID.txt NCBI_16SID.txt EzBioCloud/ezbiocloud_id_taxonomy.txt > all_mappings.txt
cat EzBioCloud/ezbiocloud_qiime_full.fasta ncbi16sID.fasta micfun.fasta > all.fasta
qiime tools import     --type FeatureData[Taxonomy]     --input-format HeaderlessTSVTaxonomyFormat     --input-path all.fasta     --output-path all_fasta.qza
qiime tools import     --type FeatureData[Taxonomy]     --input-format HeaderlessTSVTaxonomyFormat     --input-path all_uniqIDs.txt     --output-path Ids.qza
```

- Altogether, the database contains 130,122 sequences (though there might be some repetition that was overlooked)

In [ ]:
# Script for removing redundant ids
from Bio import SeqIO
import csv

exists: set = set()
mapped = open('all_uniq.fasta', 'w+')
for seq in SeqIO.parse('all.fasta', 'fasta'):
    if seq.id in exists:
        continue
    exists.add(seq.id)
    mapped.write(f'>{seq.id}\n')
    mapped.write(f'{seq.seq}\n')
mapped.close

uniq = open('all_uniqIDs.txt', 'w+')
exists2: set = set()
with open('all_mappings.txt', 'r') as i:
    for id in csv.reader(i, delimiter='\t'):
        if id[0] in exists2:
            continue
        exists2.add(id[0])
        uniq.write(f'{id[0]}\t{id[1]}\n')
uniq.close


#### Merging with other data
The qiime2 website provides links for data resources, including 16s rRNA reference sequences. These are from the SILVA and greengenes databases

In [12]:
# Import database artifacts
IDs = Artifact.load('data/artifacts/Ids.qza') 
RefSeqs = Artifact.load('data/artifacts/all_fasta.qza')
silvaSeqs = Artifact.load('data/downloads/silva-138-99-seqs.qza')
silvaIDs = Artifact.load('data/downloads/silva-138-99-tax.qza')
greenSeqs = Artifact.load('data/downloads/2022.10.backbone.full-length.fna.qza')
greenIDs = Artifact.load('data/downloads/2022.10.backbone.tax.qza')
AllIDs = ftm.merge_taxa([IDs, silvaIDs, greenIDs])
AllSeqs = ftm.merge_seqs([RefSeqs, silvaSeqs, greenSeqs])
AllIDs.merged_data.save('data/artifacts/AllIDs.qza')
AllSeqs.merged_data.save('data/artifacts/AllSeqs.qza')

/home/sc31/Bio_SDD/miniconda3/envs/qiime2-2023.2/lib/python3.8/site-packages/q2_types/feature_data/_transformer.py:258: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for id_, seq in data.iteritems():


### Taxonomic classification


# Diversity analyses

# References

# Load artifacts